In [1]:
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from sklearn.cluster import DBSCAN
from imutils.video import FPS
from imutils import paths
import dlib.cuda as cuda
import face_recognition
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os

## Main() Streaming 

In [36]:
def main():

    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    time.sleep(2.0)

    print("[INFO] clustering...")
    clt = DBSCAN(min_samples=8, metric='euclidean', eps= 0.4435, n_jobs=-1)

    # start the FPS throughput estimator
    fps = FPS().start()
    encodings = []
    labelSet = set()

    # loop over frames from the video file stream
    while True:
        # grab the frame from the threaded video stream
        frame = vs.read()
        frame = imutils.resize(frame, width=500)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        boxes = face_recognition.face_locations(rgb,model="cnn")

        encodingsSize = len(encodings)

        # compute the facial embedding for the face
        encoding = face_recognition.face_encodings(rgb, boxes)
        if encoding:
            encodings += encoding
            for i in clt.fit_predict(encodings):
                if i!=-1 :
                    labelSet.add(i)

        for i, box in enumerate(boxes):
                # coordinates
                (top, right, bottom, left) = box
                y = top - 10 if top - 10 > 10 else top + 10

                # perform classification to recognize the face
                name = "ID"
                text = "{}: {}".format(name, clt.labels_[encodingsSize+i])

                # plot at image
                cv2.rectangle(frame, (right,top), (left, bottom), (0, 0, 255), 2)
                cv2.putText(frame, text, (left, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 200), 2)

        cv2.putText(frame, 'Counting: {}'.format(len(labelSet)), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 100, 200), 2)

        
        # update the FPS counter
        fps.update()

        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

    # stop the timer and display FPS information
    fps.stop()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.stop()

In [35]:
main()

[INFO] starting video stream...
[INFO] clustering...
[INFO] elasped time: 21.31
[INFO] approx. FPS: 10.80
